In [3]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
# from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

/Users/yutikarege/Desktop/Personal Projects/Billie/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
Settings.embed_model = HuggingFaceEmbedding()

/Users/yutikarege/Desktop/Personal Projects/Billie/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
import os 
import openai
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.environ.get("API_KEY")
# openai.api_key

In [7]:
llm = OpenAI(api_key=openai.api_key)

In [8]:
response = llm.complete("What is the capital of Germany?")
print(response)

The capital of Germany is Berlin.


In [18]:
docs = '/Users/yutikarege/Desktop/Personal Projects/Billie/Lesson Plan'

In [22]:
# load some documents
documents = SimpleDirectoryReader(docs).load_data()

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./vectordb/chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("lessonplan")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Asking questions to the Lesson Plan vec db

In [20]:
llm

OpenAI(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x29132bed0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x15204dd00>, completion_to_prompt=<function default_completion_to_prompt at 0x1520b39c0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='gpt-3.5-turbo', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='sk-proj-zXB2dovgWIMQDPSDbq3FT3BlbkFJ7drUvV4y943RNRGGNIC1', api_base='https://api.openai.com/v1', api_version='')

In [21]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

# create a query engine and query
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("What is the lesson plan for Electromagnetism?")
print(response)

The lesson plan for Electromagnetism includes two content units: "Moving charges and magnetism" and "Magnetism and matter." These units are covered in the textbook chapters 4 and 5, with a total of 25 teaching periods allocated for this topic.


In [8]:
# from llama_index.llms.ollama import Ollama

# llm = Ollama(model="llama2", request_timeout=60.0)

# response = llm.complete("What is the capital of France?")
# print(response)